In [67]:
import requests
import pandas as pd
from datetime import datetime, timedelta

StatementMeta(, 59700cab-efab-4718-a36f-4c1984d62ef5, 69, Finished, Available, Finished)

In [68]:
base_url = "https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/"

bronze_table_path = "abfss://Brief_Eolienne@onelake.dfs.fabric.microsoft.com/LH_Wind_Power_Bronze.Lakehouse/Tables/dbo/wind_power"

StatementMeta(, 59700cab-efab-4718-a36f-4c1984d62ef5, 70, Finished, Available, Finished)

In [69]:
# Charger les données existantes depuis Bronze
df_spark = spark.read.format("delta").load(bronze_table_path)

# Convertir en Pandas pour manipulation plus facile
df_pandas = df_spark.toPandas()

print(f"📊 Données actuelles dans Bronze : {len(df_pandas)} lignes")

StatementMeta(, 59700cab-efab-4718-a36f-4c1984d62ef5, 71, Finished, Available, Finished)

📊 Données actuelles dans Bronze : 5184 lignes


In [70]:
# Trouver la date la plus récente dans les données
most_recent_date = pd.to_datetime(df_pandas['date'], format="%Y%m%d").max()

# Calculer le jour suivant
next_date = (most_recent_date + timedelta(days=1)).strftime("%Y%m%d")

print(f"📅 Date la plus récente : {most_recent_date.strftime('%Y-%m-%d')}")
print(f"➡️  Prochaine date à charger : {next_date}")

StatementMeta(, 59700cab-efab-4718-a36f-4c1984d62ef5, 72, Finished, Available, Finished)

📅 Date la plus récente : 2024-06-26
➡️  Prochaine date à charger : 20240627


In [71]:
# Construire l'URL du fichier CSV
file_url = f"{base_url}{next_date}_wind_power_data.csv"
print(f"🌐 URL du fichier : {file_url}")

try:
    # Télécharger le CSV depuis GitHub
    df_pandas_new = pd.read_csv(file_url)
    
    # Convertir la colonne date en datetime
    df_pandas_new['date'] = pd.to_datetime(df_pandas_new['date'])
    
    print(f"✅ Nouvelles données téléchargées : {len(df_pandas_new)} lignes")
    print(f"📊 Aperçu des nouvelles données :")
    print(df_pandas_new.head())
    
except Exception as e:
    print(f"❌ Erreur lors du téléchargement : {e}")
    print(f"💡 Cela peut signifier que le fichier pour la date {next_date} n'existe pas encore.")

StatementMeta(, 59700cab-efab-4718-a36f-4c1984d62ef5, 73, Finished, Available, Finished)

🌐 URL du fichier : https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/20240627_wind_power_data.csv
✅ Nouvelles données téléchargées : 432 lignes
📊 Aperçu des nouvelles données :
   production_id       date      time turbine_name  capacity location_name  \
0          11233 2024-06-27  00-00-00    Turbine A      2200    Location 1   
1          11234 2024-06-27  00-00-00    Turbine B      2000    Location 2   
2          11235 2024-06-27  00-00-00    Turbine C      2500    Location 3   
3          11236 2024-06-27  00-10-00    Turbine A      2200    Location 1   
4          11237 2024-06-27  00-10-00    Turbine B      2000    Location 2   

   latitude  longitude    region  status responsible_department  wind_speed  \
0   34.0522  -118.2437  Region A  Online             Operations    19.31362   
1   36.7783  -119.4179  Region B  Online             Operations    24.58335   
2   40.7128   -74.0060  Region C  Online             Operations    24.60645   
3   34.0522

In [72]:
# Convertir le DataFrame Pandas en Spark DataFrame
df_spark_new = spark.createDataFrame(df_pandas_new, schema=df_spark.schema)

# Ajouter les nouvelles données à la table Bronze (mode append)
df_spark_new.write.format('delta').mode("append").save(bronze_table_path)

print("✅ Données ajoutées avec succès dans le Lakehouse Bronze")
print(f"📊 Total après ajout : {spark.read.format('delta').load(bronze_table_path).count()} lignes")

StatementMeta(, 59700cab-efab-4718-a36f-4c1984d62ef5, 74, Finished, Available, Finished)

✅ Données ajoutées avec succès dans le Lakehouse Bronze
📊 Total après ajout : 5616 lignes
